In [1]:
import CeLEry as cel

import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.model_selection import train_test_split
from anndata import AnnData, read_h5ad

import json

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_merfish_raw = pd.read_csv("data/datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_by_gene_S1R1.csv", index_col=0)   
meta_data = pd.read_csv("data/datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_metadata_S1R1.csv", index_col=0)

data_merfish = AnnData(data_merfish_raw)

data_merfish.obs['x_cord'] = meta_data['center_x'].tolist()
data_merfish.obs['y_cord'] = meta_data['center_y'].tolist()
data_merfish_raw = data_merfish.copy()

sc.pp.filter_cells(data_merfish, min_counts=500)
sc.pp.filter_cells(data_merfish, min_genes=100)

sc.pp.neighbors(data_merfish, n_neighbors=15, use_rep='X', random_state=1)
sc.tl.louvain(data_merfish, 0.4, random_state=1)

In [3]:
Qdata = data_merfish[data_merfish.obs['x_cord'] <= np.quantile(data_merfish.obs['x_cord'], 0.5)]
Rdata = data_merfish[data_merfish.obs['x_cord'] > np.quantile(data_merfish.obs['x_cord'], 0.5)]

In [4]:
cel.get_zscore(Qdata)
cel.get_zscore(Rdata)

In [5]:
## right brain as training, left brain as testing
model_train = cel.Fit_cord (data_train = Rdata, hidden_dims = [100, 50, 20], num_epochs_max = 2000, path = "output/brain", filename = "brain_left")

100%|██████████| 2293/2293 [00:08<00:00, 284.61it/s]


Epoch:1, Loss:75.2848


100%|██████████| 2293/2293 [00:07<00:00, 311.46it/s]

Epoch:2, Loss:35.4318
Folder already exists


In [9]:
pred_cord = cel.Predict_cord (data_test = Qdata, path = "output/brain", filename = "brain_left")
pred_cord[:,0] = 1-pred_cord[:,0]

data_train = Qdata.copy()
traindata = (data_train.X.A if issparse(data_train.X) else data_train.X)
tdatax = np.expand_dims(traindata, axis = 0)
tdata_rs = np.swapaxes(tdatax, 1, 2)
test_cord = cel.wrap_gene_location(tdata_rs, data_train.obs[['x_cord', 'y_cord']])

pred_cord_transformx = pred_cord[:,0]*(test_cord.xmax-test_cord.xmin) + test_cord.xmin
pred_cord_transformy = pred_cord[:,1]*(test_cord.ymax-test_cord.ymin) + test_cord.ymin
pred_cord_transform = np.array([pred_cord_transformx, pred_cord_transformy]).T
pred_cord_transform = pd.DataFrame(pred_cord_transform)
pred_cord_transform.index = Qdata.obs.index
pred_cord_transform.columns = ['x', 'y']

pred_cord_transform.to_csv("output/brain/celery_brain_left.csv")

In [51]:
## left brain as training, right brain as testing
model_train = cel.Fit_cord (data_train = Qdata, hidden_dims = [100, 50, 20], num_epochs_max = 2000, path = "output/brain", filename = "brain_right")

In [57]:
pred_cord = cel.Predict_cord (data_test = Qdata, path = "output/brain", filename = "brain_right")
pred_cord[:,0] = 1-pred_cord[:,0]

data_train = Rdata.copy()
traindata = (data_train.X.A if issparse(data_train.X) else data_train.X)
tdatax = np.expand_dims(traindata, axis = 0)
tdata_rs = np.swapaxes(tdatax, 1, 2)
test_cord = cel.wrap_gene_location(tdata_rs, data_train.obs[['x_cord', 'y_cord']])

pred_cord_transformx = pred_cord[:,0]*(test_cord.xmax-test_cord.xmin) + test_cord.xmin
pred_cord_transformy = pred_cord[:,1]*(test_cord.ymax-test_cord.ymin) + test_cord.ymin
pred_cord_transform = np.array([pred_cord_transformx, pred_cord_transformy]).T
pred_cord_transform = pd.DataFrame(pred_cord_transform)
pred_cord_transform.index = Rdata.obs.index
pred_cord_transform.columns = ['x', 'y']

pred_cord_transform.to_csv("output/brain/celery_brain_right.csv")